In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import cholesky
from tqdm.auto import tqdm
import anDiffReg as an

In [ ]:
# exemplary data: mixture of FBMs
# this is the same system as considered in the "Non-parametric deconvolution" section of the article

ln = 100
n = 10**4
dt =  1
ts = dt*np.arange(1,ln+1)

msd = np.empty((ln-1,n), dtype = np.float64)
S = np.empty((ln,ln), dtype = np.float64)

for k in tqdm(range(n)):
    # (logD, alpha) distribution is 2 rectangles 
    if np.random.rand() < 1/2:
        alpha = np.random.uniform(0.4,0.6)
    else:
        alpha = np.random.uniform(0.8,1.0)
    logD = np.random.uniform(-1.0,1.0)

    for i in range(ln):
        for j in range(i,ln):
            S[i,j] = 10**logD * (ts[i]**alpha + ts[j]**alpha - np.abs(ts[j] - ts[i])**alpha)
            S[j,i] = S[i,j]
    A = cholesky(S, lower=False)
    xi = np.random.randn(len(ts))
    X = A.T @ xi

    msd[:,k] = np.ravel(an.tamsd(X))

In [ ]:
# fitting

ols, covOLS = an.fit_ols(msd, 1, dt)
gls, covGLS = an.fit_gls(msd, 1, dt, ols[1,:]) 

In [ ]:
# getting kde
from scipy.stats import gaussian_kde

# this is for the gls estimate 
#kde = gaussian_kde(gls, "silverman")

# for the OLS estimate switch to this line 
kde = gaussian_kde(ols, "silverman")

alphas = np.linspace(0.2,1.3,512)
logDs = np.linspace(-1.5, 1.5, 512)
X, Y = np.mgrid[-1.5:1.5:512j, 0.2:1.3:512j]
pos = np.vstack([X.ravel(), Y.ravel()])
den = np.reshape(kde(pos).T, X.shape) # original density

In [ ]:
# plotting initial estimate

fig, (ax1, ax2) = plt.subplots(1, 2, width_ratios=[2, 1])

ax1.imshow(np.rot90(den), aspect = "auto", extent = (-1.5,1.5,0.2,1.3))
ax1.set_title("Initial pdf estimate")
ax1.set_xlabel("log10 D")
ax1.set_ylabel("alpha")

denMarg0 = np.sum(den, axis=0)
denMarg0 *= 1/(sum(denMarg0)*(1.1/512))

ax2.plot(denMarg0, alphas)
ax2.set_title("Marginal distribution")
ax2.set_ylim(0.2,1.3)

In [ ]:
# simple deconvolution

dim = 1
alpha = 0.7 # α for which to deconvolve

# calculation
deconvolvedPDF1 = an.deconvolve_gls(logDs, alphas, den, dt, ln, dim, alpha, "simple")

# for the OLS switch to this lines
#w = 10 # OLS window size
#deconvolvedPDF1 = an.deconvolve_ols(logDs, alphas, den, dt, ln, dim, alpha, w, "simple")

In [ ]:
# plot

fig, (ax1, ax2) = plt.subplots(1, 2, width_ratios=[2, 1])

ax1.imshow(np.rot90(deconvolvedPDF1), aspect = "auto", extent = (-1.5,1.5,0.2,1.3))
ax1.set_title("Deconvolved pdf estimate, simple method")
ax1.set_xlabel("log10 D")
ax1.set_ylabel("alpha")

denMarg1 = np.sum(deconvolvedPDF1, axis=0)
denMarg1 *= 1/(sum(denMarg1)*(1.1/512))

ax2.plot(denMarg1, alphas)
ax2.set_title("Marginal distribution")
ax2.set_ylim(0.2, 1.3)

In [ ]:
# full deconvolution 

dim = 1
alpha_min, alpha_max = 0.3, 1.1 # range of α for which to deconvolve

# calculation, note it can take significantly longer time
#deconvolvedPDF2 = an.deconvolve_gls(logDs, alphas, den, dt, ln, dim, (alpha_min,alpha_max), "full")

# for the OLS switch to this lines
w = 10 # OLS window size
deconvolvedPDF2 = an.deconvolve_ols(logDs, alphas, den, dt, ln, dim, (alpha_min,alpha_max), w, "full")

In [ ]:
# plot

fig, (ax1, ax2) = plt.subplots(1, 2, width_ratios=[2, 1])

ax1.imshow(np.rot90(deconvolvedPDF2), aspect = "auto", extent = (-1.5,1.5,0.2,1.3))
ax1.set_title("Deconvolved pdf estimate, full method")
ax1.set_xlabel("log10 D")
ax1.set_ylabel("alpha")

denMarg2 = np.sum(deconvolvedPDF2, axis=0)
denMarg2 *= 1/(sum(denMarg2)*(1.1/512))

ax2.plot(denMarg2, alphas)
ax2.set_title("Marginal distribution")
ax2.set_ylim(0.2, 1.3)